<a href="https://colab.research.google.com/github/studio-ousia/luke/blob/master/notebooks/huggingface_open_entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reproducing experimental results of LUKE on Open Entity using Hugging Face Transformers

This notebook shows how to reproduce the state-of-the-art results on the Open Entity entity typing task reported in [this paper](https://arxiv.org/abs/2010.01057) using the Trasnsformers library and the [fine-tuned model](https://huggingface.co/studio-ousia/luke-large-finetuned-open-entity) available on the model hub.
The source code used in the experiments is also available [here](https://github.com/studio-ousia/luke/tree/master/examples/entity_typing).

In [ ]:
!gdown --id 1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im
!tar xzf /content/data.tar.gz
!cp data/OpenEntity/test.json .

Downloading...
From: https://drive.google.com/uc?id=1HlWw7Q6-dFSm9jNSCh4VaBf1PlGqt9im
To: /content/data.tar.gz
322MB [00:04, 79.1MB/s]


In [ ]:
!pip install git+https://github.com/huggingface/transformers.git@refs/pull/11223/head

  Cloning https://github.com/huggingface/transformers.git (to revision refs/pull/11223/head) to /tmp/pip-req-build-7zi9a32f
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-7zi9a32f
  Running command git fetch -q https://github.com/huggingface/transformers.git refs/pull/11223/head
  Running command git checkout -q c29e2f974b2ac3b78b4a9aaa238d49f736170dd9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 4.3MB/s 
     |████████████████████████████████| 3.3MB 18.6MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2151957 sha256=0dc9abdf36018e16050ecfc367388d46bb439d080f3b44792a69f46e4b2c4a71
  Stored in directory: /tmp/pip-ephem-wheel-cache-fnph_dsg/wheels/17/a9/0b/d1b18d07749e24c87f3e9a2045220a1cfa7d31ae75994e3270
Successfully built transformers


In [ ]:
import json
import torch
from tqdm import trange
from transformers import LukeTokenizer, LukeForEntityClassification

In [ ]:
def load_examples(dataset_file):
    with open(dataset_file, "r") as f:
        data = json.load(f)

    examples = []
    for item in data:
        examples.append(dict(
            text=item["sent"],
            entity_spans=[(item["start"], item["end"])],
            label=item["labels"]
        ))

    return examples

In [ ]:
test_examples = load_examples("test.json")

In [ ]:
model = LukeForEntityClassification.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")
model.eval()
model.to("cuda")
tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-large-finetuned-open-entity")

In [ ]:
batch_size = 128

num_predicted = 0
num_gold = 0
num_correct = 0

all_predictions = []
all_labels = []

for batch_start_idx in trange(0, len(test_examples), batch_size):
    batch_examples = test_examples[batch_start_idx:batch_start_idx + batch_size]
    texts = [example["text"] for example in batch_examples]
    entity_spans = [example["entity_spans"] for example in batch_examples]
    gold_labels = [example["label"] for example in batch_examples]

    inputs = tokenizer(texts, entity_spans=entity_spans, return_tensors="pt", padding=True)
    inputs = inputs.to("cuda")
    with torch.no_grad():
        outputs = model(**inputs)

    num_gold += sum(len(l) for l in gold_labels)
    for logits, labels in zip(outputs.logits, gold_labels):
        for index, logit in enumerate(logits):
            if logit > 0:
                num_predicted += 1
                predicted_label = model.config.id2label[index]
                if predicted_label in labels:
                    num_correct += 1

precision = num_correct / num_predicted
recall = num_correct / num_gold
f1 = 2 * precision * recall / (precision + recall)

print(f"\n\nprecision: {precision} recall: {recall} f1: {f1}")

100%|██████████| 16/16 [01:12<00:00,  4.53s/it]



precision: 0.7980295566502463 recall: 0.7657563025210085 f1: 0.781559903511123
